__собираем предложения о продаже домов__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/house/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_house.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/house/avito/avito_2022-09-15_11-54_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-42_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-47_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-21_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-22_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-24_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-09-19_12-40_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-09-22_09-59_raw_sudak.xlsx']

In [7]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

5830


In [8]:
# data_avito

In [9]:
from lib.avito import AvitoDataCleanerRealtyHouse

data_avito = AvitoDataCleanerRealtyHouse().transform( data_avito )

print(len(data_avito))

5830


In [10]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
1597,2520993781,дом 54 м² на участке 4 сот.,4750000,,"садоводческое товарищество Рассвет-2, 2","Продам дом 54 кв. М возле моря на ровном участке 4 сотки. Отличное расположение — место тупиковое, не проездное, а значит ни пыли, ни шума! До остановки 5 мин и к морю 5 мин! В шаговой доступности магазин с широким ассортиментом. \n\nДом обустроен для жилья,",33,2022-09-15 11:53:29.976,sevastopol,54.00,False,False,True,4.00,4.75,2-4,50-70
5399,2417871765,дом 50 м² на участке 25 сот.,3900000,,с. Репино,"Идеально для сельского хозяйства. Экологически чистое место. Вокруг куча места. По документам 25 соток, фактически 30 соток. Огромный сад с произростанием всего что можно представить. Есть своя пасика. ( Могу оставить будущему владельцу). Две скважины, све",5,2022-09-16 15:22:23.263,bahchisaray,50.00,False,False,False,25.00,3.90,20+,30-50
2001,2425366717,"дача 20 м² на участке 4,3 сот.",2600000,,садоводческое товарищество Наука,"Продам Недорого дачу / земельный участок в Тсн Наука, мыс Фиолент, район Автобата, напротив бывшего Авторынка. Вторая Линия От Леса. \nКлассное Тихое Место Для Шашлыков На Даче. \nУлица 7-я бригада, участок 175. \nКадастровый номер 91:02:006007:156.\nКооперати",42,2022-09-15 11:53:29.976,sevastopol,20.00,False,False,True,4.30,2.60,4-8,<30


In [11]:
data_avito['place'].unique()

array(['sevastopol', 'bahchisaray', 'sudak'], dtype=object)

In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5830 entries, 0 to 5829
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   avito_id             5830 non-null   int64         
 1   title                5830 non-null   object        
 2   price                5830 non-null   int64         
 3   obj_name             5830 non-null   object        
 4   adr                  5830 non-null   object        
 5   description          5830 non-null   object        
 6   avito_page           5830 non-null   int64         
 7   ts                   5830 non-null   datetime64[ns]
 8   place                5830 non-null   object        
 9   house_area           5830 non-null   float64       
 10  is_part              5830 non-null   bool          
 11  is_townhouse         5830 non-null   bool          
 12  is_SNT               5830 non-null   bool          
 13  land_area            5830 non-nul

In [14]:
data_avito.isnull().any()

avito_id               False
title                  False
price                  False
obj_name               False
adr                    False
description            False
avito_page             False
ts                     False
place                  False
house_area             False
is_part                False
is_townhouse           False
is_SNT                 False
land_area              False
priceM                 False
land_size_category      True
house_size_category     True
dtype: bool

In [15]:
data_avito[ data_avito['land_size_category'].isnull() ]

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
2283,2337799298,,11900000,,Восточная ул.,"Предлагаю 1/2 дома общей площадью 120 кв. М. В центре города в отличном районе для проживания на земельном участке 1.6 сотки. Вся инфраструктура рядом, транспортные развязки в шаговой доступности. При этом дом расположен в тихом месте, с каждого этажа дома",47,2022-09-15 11:53:29.976,sevastopol,0.00,True,False,False,0.00,11.90,NaN,NaN
2614,2034310938,,5000000,,"с. Танковое, ул. Ходырева","Собственник. Срочно продам дом 74,3 кв м на земельном участке 814 м кв. Санузел и туалет в доме, 2 спальни. 1965 года постройки, камень бут, очень тёплый зимой и прохладный летом. Газ, приборы учета на все коммуникации. В дом есть 2 входа. Также на участке",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,False,0.00,5.00,NaN,NaN
2624,2391096190,,3000000,,"с. Растущее, Садовая ул., 8","Продам Дом 75 КВ. М, 10 Соток Земли Ижс Продам дом в живописном районе Крыма, в селе Растущее 10 соток земли, 75 кв. М, 4 комнаты, коммуникации: свет, газ, вода, скважина, отопление Агв. Дом чистый, уютный, заходи живи. Участок ровный, сад, весь обработанн",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,True,0.00,3.00,NaN,NaN
2733,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,1,2022-09-15 14:47:45.056,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN
5076,2337799298,,11900000,,Восточная ул.,"Предлагаю 1/2 дома общей площадью 120 кв. М. В центре города в отличном районе для проживания на земельном участке 1.6 сотки. Вся инфраструктура рядом, транспортные развязки в шаговой доступности. При этом дом расположен в тихом месте, с каждого этажа дома",48,2022-09-16 15:21:10.419,sevastopol,0.00,True,False,False,0.00,11.90,NaN,NaN
5213,2391096190,,3000000,,"с. Растущее, Садовая ул., 8","Продам Дом 75 КВ. М, 10 Соток Земли Ижс Продам дом в живописном районе Крыма, в селе Растущее 10 соток земли, 75 кв. М, 4 комнаты, коммуникации: свет, газ, вода, скважина, отопление Агв. Дом чистый, уютный, заходи живи. Участок ровный, сад, весь обработанн",1,2022-09-16 15:22:23.263,bahchisaray,0.00,False,False,True,0.00,3.00,NaN,NaN
5384,2034310938,,5000000,,"с. Танковое, ул. Ходырева","Собственник. Срочно продам дом 74,3 кв м на земельном участке 814 м кв. Санузел и туалет в доме, 2 спальни. 1965 года постройки, камень бут, очень тёплый зимой и прохладный летом. Газ, приборы учета на все коммуникации. В дом есть 2 входа. Также на участке",4,2022-09-16 15:22:23.263,bahchisaray,0.00,False,False,False,0.00,5.00,NaN,NaN
5511,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,1,2022-09-16 15:24:24.472,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN
5645,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,2,2022-09-19 12:40:34.305,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN
5782,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,2,2022-09-22 09:59:12.841,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN


---

In [16]:
data_avito['url'] = 'https://avito.ru/' + data_avito['avito_id'].astype(int).astype(str)
data_avito = data_avito.drop(columns=['avito_id'])
# data_cian = data_cian.rename(columns={'cian_url':'url'})

In [17]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

5830


## обновляем таблицу адресов

In [18]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [19]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data[['adr_orig','adr']].sample(10)

  0%|          | 0/5830 [00:00<?, ?it/s]

  0%|          | 0/5830 [00:00<?, ?it/s]

,adr_orig,adr
3462,садоводческое товарищество Бриг-2,"Севастополь,садоводческое товарищество Бриг-2"
1388,"садоводческое товарищество Икар-1, 138","Севастополь,садоводческое товарищество Икар-1, 138"
743,"ул. Челнокова, 15/34","Севастополь,ул. Челнокова, 15/34"
2355,"с. Осипенко, Багряный пер.","Севастополь,с. Осипенко, Багряный пер."
2216,"Готская ул., 84Б","Севастополь,Готская ул., 84Б"
797,район Северная Сторона,"Севастополь,район Северная Сторона"
3974,Сапунгорская ул.,"Севастополь,Сапунгорская ул."
3032,ул. Генерала Мельника,"Севастополь,ул. Генерала Мельника"
445,"ул. Военных Строителей, 3Б/12","Севастополь,ул. Военных Строителей, 3Б/12"
5800,"с. Миндальное, СОТ Капсель","Судак,с. Миндальное, СОТ Капсель"


In [20]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[0])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

if len(loc_undef)>0:
    # определяем координаты по строке адреса
    loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
    logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')
    loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 
    assert len(loc)>0
    loc.to_pickle(loc_file_path)

[INFO    ] 2022-09-23 18:08:57 | LocationUpdater: 6042 addresses in index
[INFO    ] 2022-09-23 18:08:57 | LocationUpdater: 6042 addresses total
[INFO    ] 2022-09-23 18:08:57 | LocationUpdater: 6042 addresses defined
[INFO    ] 2022-09-23 18:08:57 | LocationUpdater: 0 addresses undefined


In [21]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

## дополняем данные геометкой

In [22]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

всего записей: 5830
записей без геометки: 0


In [23]:
# data.loc[ data['adr_orig']=='квартал Старый Город', ['latitude','longitude']] 
data[ data['adr_orig'].str.match('.*садоводческое товарищество Голубой залив.*') ][['adr','latitude','longitude']]

,adr,latitude,longitude
49,"Севастополь,садоводческое товарищество Голубой залив",55.518499,49.066574
347,"Севастополь,садоводческое товарищество Голубой залив, 96",44.547851,33.429052
689,"Севастополь,садоводческое товарищество Голубой залив, 19",44.547286,33.429815
1797,"Севастополь,садоводческое товарищество Голубой залив, 115",44.548641,33.429348
2009,"Севастополь,садоводческое товарищество Голубой залив, 115",44.548641,33.429348
2923,"Севастополь,садоводческое товарищество Голубой залив",55.518499,49.066574
3163,"Севастополь,садоводческое товарищество Голубой залив, 96",44.547851,33.429052
3628,"Севастополь,садоводческое товарищество Голубой залив, 19",44.547286,33.429815
4610,"Севастополь,садоводческое товарищество Голубой залив, 115",44.548641,33.429348
4860,"Севастополь,садоводческое товарищество Голубой залив, 115",44.548641,33.429348


In [24]:
data.loc[data['adr_orig']=='садоводческое товарищество Голубой залив', ['latitude','longitude']]=None
# data.loc[data['adr_orig'].str.match('.*садоводческое товарищество Голубой залив.*'),['latitude','longitude']]=None
data.loc[data['adr_orig']=='квартал Старый Город',['latitude','longitude']]=None

In [25]:
assert len(data)>0
data.to_pickle(result_file_path)

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5830 entries, 0 to 5829
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   title                5830 non-null   object        
 1   price                5830 non-null   int64         
 2   obj_name             5830 non-null   object        
 3   adr_orig             5830 non-null   object        
 4   description          5830 non-null   object        
 5   avito_page           5830 non-null   int64         
 6   ts                   5830 non-null   datetime64[ns]
 7   place                5830 non-null   object        
 8   house_area           5830 non-null   float64       
 9   is_part              5830 non-null   bool          
 10  is_townhouse         5830 non-null   bool          
 11  is_SNT               5830 non-null   bool          
 12  land_area            5830 non-null   float64       
 13  priceM               5830 non-nul